In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import pandas as pd

from Latent_Dirichlet_Allocation import normalization_string, create_stop_word
from Latent_Dirichlet_Allocation import LDA_In_EM, LDA_In_VB, LDA_In_CGS

%matplotlib inline

In [2]:
df_tweet1 = pd.read_csv('csv_data/20250127_tweet_data.csv')
df_tweet1['text'] = df_tweet1['text'].str.removeprefix('[').str.removesuffix(']')
df_tweet2 = pd.read_csv('csv_data/20250304_tweet_data.csv')
df_tweet2['text'] = df_tweet2['text'].str.removeprefix('[').str.removesuffix(']')
df_tweet = pd.concat([df_tweet1, df_tweet2]).reset_index(drop=True)
df_tweet = df_tweet[['text']]
df_tweet = normalization_string(df_tweet, 'text')
df_tweet

,text
0,'昨日は山でメガネを失くし、今日は脚立登ってたら腰を痛めた。今メガネ買いにショッピングモール...
1,'フードコートで休憩すると向かいのjinsのチョンウォヌデカ広告を見ながらご飯食えていいのよね'
2,'入り口入ったらウォヌがお出迎え:heart_exclamation:ウォヌを抜擢してくれた...
3,"'眼鏡どこで作ろかな...', '', 'zoffかjinsくらいしかおもいうかばん'"
4,'jinsは当日だったんだよ:persevering_face:'
...,...
3156,'フレームだけ新古品2000円で買ってjinsでレンズ交換:upside-down_face...
3157,'#マスクをしたらレンズが曇っちゃうを選んだあなたには、くもり止めレンズ搭載のjins pr...
3158,'わたしはメガネを#4本以上持っています:glasses::glasses::glasses...
3159,'#どうしても隙間から花粉が入ってくると思っているあなたには、花粉を99%カットしてくれるj...


In [3]:
with open('json_data/custom_stopwords_en.json', 'r') as f:
	stop_word_en = json.load(f)
	stop_word_en = stop_word_en['stopwords']

with open('json_data/custom_stopwords_ja.json', 'r') as f:
	stop_word_ja = json.load(f)
	stop_word_ja = stop_word_ja['stopwords']

stop_word_digit1 = [str(idx) for idx in range(10000)]
stop_word_digit2 = [str(idx).zfill(2) for idx in range(100)]
stop_word_digit3 = [str(idx).zfill(3) for idx in range(1000)]
stop_word_digit4 = [str(idx).zfill(4) for idx in range(10000)]
stop_word_alpha  = [chr(idx) for idx in range(ord('a'), ord('z')+1)]
stop_word_ALPHA  = [chr(idx) for idx in range(ord('A'), ord('Z')+1)]
stop_word_hira   = [chr(idx) for idx in range(ord('あ'), ord('ん')+1)]
stop_word_kata   = [chr(idx) for idx in range(ord('ァ'), ord('ン')+1)]
stop_word_greece = [chr(idx) for idx in range(ord('α'), ord('ω')+1)]
stop_word_GREECE = [chr(idx) for idx in range(ord('Α'), ord('Ω')+1)]
stop_word_cyril  = [chr(idx) for idx in range(ord('а'), ord('я')+1)]
stop_word_CYRIL  = [chr(idx) for idx in range(ord('А'), ord('Я')+1)]
stop_word_symbol = ['・', '゚']
stop_word_custom = stop_word_digit1 + stop_word_digit2 + stop_word_digit3 + stop_word_digit4\
    				+ stop_word_alpha + stop_word_ALPHA\
            		+ stop_word_hira  + stop_word_kata\
                  	+ stop_word_greece + stop_word_GREECE\
                    + stop_word_cyril + stop_word_CYRIL\
                    + stop_word_symbol

stop_word = create_stop_word(df_tweet, 'text', stop_word_en + stop_word_ja + stop_word_custom, 5)

In [4]:
model = LDA_In_EM(df_tweet, stop_word=stop_word, topic_num=5)
model.fit()

print(f'topic of EM algorithm: finished')

学習回数：0
誤差：32.834302457544446

学習回数：100
誤差：0.19700117765136835

学習回数：200
誤差：0.004101431471646557

学習回数：300
誤差：0.006122027037321474

学習回数：400
誤差：0.0011918145923388104

学習回数：500
誤差：0.002108403866946979

学習回数：600
誤差：0.0029892266081118554

学習回数：700
誤差：0.014788141324482978

学習回数：800
誤差：6.293669070771716e-06

学習回数：900
誤差：2.0499570131052616e-05

学習回数：1000
誤差：0.015344418828403626

学習回数：1100
誤差：0.0006040646749783012

学習回数：1200
誤差：0.0022081440389696734

学習回数：1300
誤差：1.1199447875875071e-05

topic of EM algorithm: finished


In [5]:
pd_θ, pd_Φ = model.stats_info()
pd_θ.to_parquet('./parquet_data/20250227_tweet_EM_θ.parquet')
pd_Φ.to_parquet('./parquet_data/20250227_tweet_EM_Φ.parquet')

In [6]:
model = LDA_In_VB(df_tweet, stop_word=stop_word, topic_num=5)
model.fit()

print(f'topic of Variational Bayes algorithm: finished')

学習回数：0
誤差：8620854.313870367

学習回数：100
誤差：0.18622315976021847

topic of Variational Bayes algorithm: finished


In [7]:
pd_θ, pd_Φ = model.stats_info()
pd_θ.to_parquet('./parquet_data/20250227_tweet_VB_θ.parquet')
pd_Φ.to_parquet('./parquet_data/20250227_tweet_VB_Φ.parquet')

In [8]:
model = LDA_In_CGS(df_tweet, stop_word=stop_word, topic_num=5)
model.fit()

print(f'topic of Variational Bayes algorithm: finished')

学習回数：0
誤差：49.59223846118444

学習回数：100
誤差：0.049874592848769383

学習回数：200
誤差：0.033737468059057085

学習回数：300
誤差：0.04531310656784593

学習回数：400
誤差：0.020898480216646753

topic of Variational Bayes algorithm: finished


In [9]:
pd_θ, pd_Φ = model.stats_info()
pd_θ.to_parquet('./parquet_data/20250227_tweet_CGS_θ.parquet')
pd_Φ.to_parquet('./parquet_data/20250227_tweet_CGS_Φ.parquet')